In [9]:
import numpy 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import operator
import math
import random

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [10]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(math.sin, 1)
pset.addTerminal(1)
pset.addTerminal(3)
pset.renameArguments(ARG0='x')

In [11]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [12]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
sample = numpy.random.uniform(low=-10.0, high=10.0, size=(200,))

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = []
    for point in points:
        actual = None
        if point > 0 :
            actual = (1 / point) + math.sin(point)
        else:
            actual = (2*point) + (point*point) + 3.0
        sqerrors.append((func(point) - actual)**2)

    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=sample)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [13]:
def main(seedno):
    random.seed(seedno)

    hof = tools.HallOfFame(1)
    pop = toolbox.population(n=1000)
    cxpb = 0.85
    mutpb = 0.15
    ngen = 50
    mu = 1000
    lam = 990

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaMuPlusLambda(pop, toolbox,mu, lam, cxpb, mutpb, ngen, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof

In [14]:
bests = []
for _ in range(3):
    pop, log, hof = main(_)
    bests.append(hof[0])
    

In [15]:
for best in bests:
    print(best)
    


In [16]:
for best in bests:
    print(best.fitness)